In [1]:
!pip install openpyxl
!pip install pdfplumber
!pip install transformers


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#loading libraries
import pdfplumber
import requests
from transformers import pipeline, AutoTokenizer

#download file
def downloadFiles(url, name):
    response = requests.get(url)
    pdf_file = open(name + ".pdf", "wb")
    pdf_file.write(response.content)
    pdf_file.close()

c:\Users\sumed\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\sumed\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\onnx\_internal\_beartype.py:30: UserWarning: unhashable type: 'list'
  warnings.warn(f"{e}")


In [3]:
#files with file url, name, page_number and device_name
files = []
files.append({"url": "http://www.ti.com/lit/ds/symlink/lm46000-q1.pdf", "name" : "LM46000QPWPTQ1", "page" : 46, "device_name"  : "LM46000-Q1"})
files.append({"url": "http://www.ti.com/lit/ds/symlink/lm46001-q1.pdf", "name" : "LM46001QPWPTQ1", "page" : 49, "device_name"  : "LM46001-Q1"})
files.append({"url": "http://www.ti.com/lit/ds/symlink/lm46002-q1.pdf", "name" : "LM46002QPWPTQ1", "page" : 48, "device_name"  : "LM46002-Q1"})
files.append({"url": "http://www.ti.com/lit/ds/symlink/lm5165.pdf", "name" : "LM5165XDRCT", "page" : 3, "device_name"  : "LM5165"})
files.append({"url": "http://www.ti.com/lit/ds/symlink/lm53600-q1.pdf", "name" : "LM53600AQDSXTQ1", "page" : 3, "device_name"  : "LM53600-Q1"})
files.append({"url": "http://www.ti.com/lit/ds/symlink/lm74610-q1.pdf", "name" : "LM74610QDGKTQ1", "page" : 25, "device_name"  : "LM74610-Q1"})

In [4]:
#querying model
def answerQuery(text, query, question_answerer):
    context = text
    result = question_answerer(question=query,context=context)
    return {"Answer" : result['answer'], "score": round(result['score'], 4), "start": result['start'], "end": result['end']}

In [5]:
#trimming result
def processResult(res, query):
    res.sort(key=lambda x: x['score'], reverse=True)
    result = {"query": query, "maxRes": max(res, key=lambda x: x['score']), "top10Res": res[:10]}
    return result

In [6]:
#process files
def processFile(pdf_file, num_pages, line_threshold, question_answerer, queries):
    text = ""
    results = []

    with pdfplumber.open(pdf_file) as pdf:
        pages = pdf.pages[0:num_pages]  #Only using first page in my instance
        for query in queries:
            result = []
            for page in pages:
                lines = page.extract_text().split("\n")
                text = ""
                line_thres = 1
                for line in lines:
                    text += line + "\n"
                    if line_thres == line_threshold:
                        line_thres = 1
                        result.append(answerQuery(text, query, question_answerer))
                        text = ""
                    else:
                        line_thres += 1
            
            results.append(processResult(result, query))
    return results

In [7]:
#load_model
question_answerer = pipeline("question-answering", model='google-bert/bert-large-uncased-whole-word-masking-finetuned-squad', device="cuda")

#params
line_threshold = 10     # this can be changed for fine tuning of results
num_pages = 24           # this can be set upto total number of document pages

allResults = []

#path to .pdf file
for file in files:
    #queries
    queries = []
    queries.append("What is the Brand of " + file['device_name'] + "?")
    queries.append("What is the Regulator Function of " + file['device_name'] + "?")
    queries.append("What is the Input Voltage Range of " + file['device_name'] + "?")
    queries.append("What is the Output Voltage Range of " + file['device_name'] + "?")
    queries.append("What is the Maximum Output Current of " + file['device_name'] + "?")
    queries.append("What are the total Number of Outputs of " + file['device_name'] + "?")
    queries.append("What is the Mounting Type of " + file['device_name'] + "?")
    queries.append("What is the Package Type of " + file['device_name'] + "?")
    queries.append("How many pin device is " + file['device_name'] + "?")
    queries.append("What is the Output Type of " + file['device_name'] + "?")
    queries.append("What is the Maximum Switching Frequency of " + file['device_name'] + "?")
    #queries.append("What kind of Switching Regulator is in " + file['device_name'] + "?")
    queries.append("What is the Maximum Efficiency of " + file['device_name'] + "?")
    queries.append("What is the Operating Temperature Range of " + file['device_name'] + "?")
    queries.append("What are the Dimensions of " + file['device_name'] + "?")
    queries.append("What are the Package Dimensions of " + file['device_name'] + "?")
    queries.append("What is the Minimum Output Voltage of " + file['device_name'] + "?")
    queries.append("What is the Minimum Input Voltage of " + file['device_name'] + "?")
    queries.append("What is the Length of " + file['device_name'] + "?")
    queries.append("What is the Width of " + file['device_name'] + "?")
    queries.append("What is the Height of " + file['device_name'] + "?")
    queries.append("What is the Minimum Operating Temperature of " + file['device_name'] + "?")
    queries.append("What is the Maximum Operating Temperature of " + file['device_name'] + "?")
    queries.append("What is the Maximum Input Voltage of " + file['device_name'] + "?")
    queries.append("What is the Maximum Output Voltage of " + file['device_name'] + "?")

    downloadFiles(file['url'], file['name'])

    pdf_file = r"%s.pdf" % (file['name'])
    
    fileResult = processFile(pdf_file, num_pages, line_threshold, question_answerer, queries)
    allResults.append({"file" : file['name'], "result" : fileResult})


Some weights of the model checkpoint at google-bert/bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
c:\Users\sumed\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\pipelines\base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [8]:
allResults

[{'file': 'LM46000QPWPTQ1',
  'result': [{'query': 'What is the Brand of LM46000-Q1?',
    'maxRes': {'Answer': 'ycneiciffE', 'score': 0.9928, 'start': 4, 'end': 14},
    'top10Res': [{'Answer': 'ycneiciffE',
      'score': 0.9928,
      'start': 4,
      'end': 14},
     {'Answer': 'ycneiciffE', 'score': 0.9904, 'start': 4, 'end': 14},
     {'Answer': 'mhom', 'score': 0.9884, 'start': 61, 'end': 65},
     {'Answer': 'TUOV', 'score': 0.9873, 'start': 0, 'end': 4},
     {'Answer': 'ycneiciffE', 'score': 0.9862, 'start': 21, 'end': 31},
     {'Answer': 'tnerruC\nnwodtuhS', 'score': 0.9851, 'start': 12, 'end': 28},
     {'Answer': 'ProductFolderLinks',
      'score': 0.9834,
      'start': 193,
      'end': 211},
     {'Answer': 'ycneiciffE', 'score': 0.9777, 'start': 66, 'end': 76},
     {'Answer': 'QI', 'score': 0.971, 'start': 0, 'end': 2},
     {'Answer': 'egakaeL', 'score': 0.9709, 'start': 0, 'end': 7}]},
   {'query': 'What is the Regulator Function of LM46000-Q1?',
    'maxRes': {'

In [9]:
import json

# Convert the array to JSON format
json_array = json.dumps(allResults)

# Write the JSON to a file
with open('queries.json', 'w') as json_file:
    json_file.write(json_array)
    json_file.close()

In [10]:
import re

def getTempRange(item):
    temps = []
    top10res = item['top10Res']
    for res in top10res:
        # The regular expression pattern
        if "°C" in res['Answer']:
            if "to" in res['Answer']:
                temps.append({"Answer" : res['Answer'], "score" : res['score']})
    if len(temps) > 0:
        return max(temps, key=lambda x: x['score'])['Answer']
    else:
        return item['maxRes']['Answer']

def getVoltageRange(item):
    temps = []
    top10res = item['top10Res']
    for res in top10res:
        # The regular expression pattern
        if "V" in res['Answer']:
            if "to" in res['Answer']:
                temps.append({"Answer" : res['Answer'], "score" : res['score']})
    if len(temps) > 0:
        return max(temps, key=lambda x: x['score'])['Answer']
    else:
        return item['maxRes']['Answer']

def getVoltage(item):
    temps = []
    top10res = item['top10Res']
    for res in top10res:
        # The regular expression pattern
        if "V" in res['Answer']:
            temps.append({"Answer" : res['Answer'], "score" : res['score']})
    if len(temps) > 0:
        return max(temps, key=lambda x: x['score'])['Answer']
    else:
        return item['maxRes']['Answer']

def getMinTemperature(item):
    temps = []
    top10res = item['top10Res']
    for res in top10res:
        pattern = r'(?:.*?°C){1}'

        # Using re.search to find the pattern in the text
        match = re.match(pattern, res['Answer'])
        # The regular expression pattern
        if match:
            temps.append({"Answer" : res['Answer'], "score" : res['score']})
    if len(temps) > 0:
        ans = max(temps, key=lambda x: x['score'])['Answer']
        if "to" in ans:
            ans = ans.split('to')[0]
        return ans
    else:
        return item['maxRes']['Answer']

def getMaxTemperature(item):
    temps = []
    top10res = item['top10Res']
    for res in top10res:
        pattern = r'(?:.*?°C){1}'

        # Using re.search to find the pattern in the text
        match = re.match(pattern, res['Answer'])
        # The regular expression pattern
        if match:
            temps.append({"Answer" : res['Answer'], "score" : res['score']})
    if len(temps) > 0:
        ans = max(temps, key=lambda x: x['score'])['Answer']
        if "to" in ans:
            ans = ans.split('to')[1]
        return ans
    else:
        return item['maxRes']['Answer']
    
def getCurrent(item):
    temps = []
    top10res = item['top10Res']
    for res in top10res:
        # The regular expression pattern
        if "A" in res['Answer'] or "Ω" in res['Answer']:
            temps.append({"Answer" : res['Answer'], "score" : res['score']})
    if len(temps) > 0:
        return max(temps, key=lambda x: x['score'])['Answer']
    else:
        return item['maxRes']['Answer']

def getFrequency(item):
    temps = []
    top10res = item['top10Res']
    for res in top10res:
        # The regular expression pattern
        if "Hz" in res['Answer']:
            temps.append({"Answer" : res['Answer'], "score" : res['score']})
    if len(temps) > 0:
        ans = max(temps, key=lambda x: x['score'])['Answer']
        return re.match(r"(.*Hz)", ans).group()
    else:
        return item['maxRes']['Answer']

def getEfficiency(item):
    temps = []
    top10res = item['top10Res']
    for res in top10res:
        # The regular expression pattern
        if "%" in res['Answer']:
            temps.append({"Answer" : res['Answer'], "score" : res['score']})
    if len(temps) > 0:
        return max(temps, key=lambda x: x['score'])['Answer']
    else:
        return item['maxRes']['Answer']

def getDimensions(item):
    temps = []
    top10res = item['top10Res']
    for res in top10res:
        # The regular expression pattern
        pattern = r'(?:.*?mm){3}'

        # Using re.search to find the pattern in the text
        match = re.search(pattern, res['Answer'])
        
        if match:
            temps.append({"Answer" : res['Answer'], "score" : res['score']})
    if len(temps) > 0:
        return max(temps, key=lambda x: x['score'])['Answer']
    else:
        return item['maxRes']['Answer']

In [11]:
import json

manufacturer = []

attributes = []

values = []
with open('queries.json', 'r') as f:
    data = json.load(f)
    for item in data:
        attributes.append("Brand")
        attributes.append("Regulator Function")
        attributes.append("Input Voltage Range")
        attributes.append("Output Voltage Range")
        attributes.append("Maximum Output Current")
        attributes.append("Number of Outputs")
        attributes.append("Mounting Type")
        attributes.append("Package Type")
        attributes.append("Pin Count")
        attributes.append("Output Type")
        attributes.append("Maximum Switching Frequency")
        #attributes.append("Switching Regulator")
        attributes.append("Efficiency")
        attributes.append("Operating Temperature Range")
        attributes.append("Dimensions")
        attributes.append("Package Dimensions")
        attributes.append("Minimum Output Voltage")
        attributes.append("Minimum Input Voltage")
        attributes.append("Length")
        attributes.append("Width")
        attributes.append("Height")
        attributes.append("Minimum Operating Temperature")
        attributes.append("Maximum Operating Temperature")
        attributes.append("Maximum Input Voltage")
        attributes.append("Maximum Output Voltage")

        minMax = False
        for res in item['result']:
            manufacturer.append(item['file'])
            if "Operating Temperature Range" in res['query']:
                tempRange = getTempRange(res)
                if "to" in tempRange:
                    minMax = True
                    minTemp = tempRange.split("to")[0]
                    maxTemp = tempRange.split("to")[1]
                values.append(tempRange)
            elif "Voltage Range" in res['query']:
                values.append(getVoltageRange(res))
            elif "Output Current" in res['query']:
                values.append(getCurrent(res))
            elif "Maximum Switching Frequency" in res['query']:
                values.append(getFrequency(res))
            elif "Maximum Efficiency" in res['query']:
                values.append(getEfficiency(res))
            elif "Dimensions" in res['query']:
                values.append(getDimensions(res))
            elif "Output Voltage" in res['query']:
                values.append(getVoltage(res))
            elif "Input Voltage" in res['query']:
                values.append(getVoltage(res))
            elif "Minimum Operating Temperature" in res['query']:
                if minMax == True:
                    values.append(minTemp)
                else:
                    values.append(getMaxTemperature(res))
            elif "Maximum Operating Temperature" in res['query']:
                if minMax == True:
                    values.append(maxTemp)
                else:
                    values.append(getMaxTemperature(res))
            else:
                values.append(res['maxRes']['Answer'])


In [12]:
print(len(manufacturer))
print(len(attributes))
print(len(values))

144
144
144


In [13]:
import pandas as pd

df = pd.DataFrame(data={'Manufacturer Part Number' : manufacturer, 'Attribute' : attributes, 'Attribute Value' : values })

In [14]:
print(df.head(46))

   Manufacturer Part Number                      Attribute     Attribute Value
0            LM46000QPWPTQ1                          Brand          ycneiciffE
1            LM46000QPWPTQ1             Regulator Function         Temperature
2            LM46000QPWPTQ1            Input Voltage Range       3.5 V to 60 V
3            LM46000QPWPTQ1           Output Voltage Range       3.5 V to 60 V
4            LM46000QPWPTQ1         Maximum Output Current               0.5 A
5            LM46000QPWPTQ1              Number of Outputs               7.3.6
6            LM46000QPWPTQ1                  Mounting Type                   A
7            LM46000QPWPTQ1                   Package Type              AddedR
8            LM46000QPWPTQ1                      Pin Count                  29
9            LM46000QPWPTQ1                    Output Type         Temperature
10           LM46000QPWPTQ1    Maximum Switching Frequency             500 kHz
11           LM46000QPWPTQ1                     Effi

In [15]:
df.to_excel("Assignment - AS-AIML-01 - Output File.xlsx", index=False)